# USER INPUT

**Paste the Excel Line in the Cell Below!**

In [34]:
params = {
"event_id": "nc72282711",
"geography": "tract",
"intensity_metric": "min",
"utility_loss_rate": 0.20
}

# CODE BASE




In [35]:
EVENTID = params["event_id"]

In [36]:
import os
import pandas as pd
import numpy as np


In [37]:
'''# Mount the Google Drive
from google.colab import drive
drive.mount('/content/drive')'''

"# Mount the Google Drive\nfrom google.colab import drive\ndrive.mount('/content/drive')"

In [38]:
'''# Setting System to Import Packages
import sys
sys.path.append('/content/drive/MyDrive/capstone/WorkingScripts')
parent_dir = '/content/drive/MyDrive/capstone/'''

"# Setting System to Import Packages\nimport sys\nsys.path.append('/content/drive/MyDrive/capstone/WorkingScripts')\nparent_dir = '/content/drive/MyDrive/capstone/"

In [39]:
parent_dir = os.path.dirname(os.getcwd())

# SKIP


In [40]:
# Install Required Packages
import environment_setting as es # Replace my_module with the actual name of your module

es.setting_env(parent_dir)

Environment is set up successfully.


In [41]:
# Running o1
import o1_getshakemap as getshakemap

feed_url = getshakemap.FEEDURL.format(EVENTID)
jdict = getshakemap.fetch_earthquake_data(feed_url=feed_url)

event = getshakemap.retrieve_event_data(jdict)
getshakemap.download_and_extract_shakemap(event)

Extracted ShakeMap for event nc72282711 to /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Gradschool/4_SPRING25/Capstone/github/EarthquakeDamageModel_Heinz/ShakeMaps/nc72282711
Success! Downloaded ShakeMap for event nc72282711...


'/Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Gradschool/4_SPRING25/Capstone/github/EarthquakeDamageModel_Heinz/ShakeMaps/nc72282711'

In [42]:
# Running o2 census intersect
import o2_census_intersect as census

event_dir = os.path.join(parent_dir, 'ShakeMaps', EVENTID)

census.shakemap_into_census_geo(event_dir, parent_dir)

# Update with the actual path
GPKG_PATH = os.path.join(event_dir, "eqmodel_outputs.gpkg")


Census tracts shapefile already exists /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Gradschool/4_SPRING25/Capstone/github/EarthquakeDamageModel_Heinz/Data/merged_shapefile/Nationwide_Tracts.gpkg
---GeoPackage already exists: /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Gradschool/4_SPRING25/Capstone/github/EarthquakeDamageModel_Heinz/ShakeMaps/nc72282711/eqmodel_outputs.gpkg
Saved shakemap_tractclip_pga (clipped ShakeMap to tracts) to /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Gradschool/4_SPRING25/Capstone/github/EarthquakeDamageModel_Heinz/ShakeMaps/nc72282711/eqmodel_outputs.gpkg
=======================  pga clipped ====
      AREA  PERIMETER  PGAPOL_  PGAPOL_ID  GRID_CODE  PARAMVALUE STATEFP  \
0      0.0        0.0        0          0          0        0.01      06   
1      0.0        0.0        0          0          0        0.01      06   
2      0.0        0.0        0          0          0        0.01      06   
3      0.0        0.

# RUN

In [43]:
# Running o3 clip building
import o3_clip_eventdata_buildingstocks as bs
o3output = bs.building_clip_analysis(parent_dir, EVENTID)

1. Reading event data for event ID: nc72282711
2. Reading building count data...
3. Reading building stock data...
Building stock data exists at /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Gradschool/4_SPRING25/Capstone/github/EarthquakeDamageModel_Heinz/Tables/Building_Percentages_Per_Tract_ALLSTATES.csv
4. Merging building count and building stock data...
5. Merging event data with building data...
            GEOID  max_intensity  min_intensity  mean_intensity  \
0     06001400100           0.02           0.02            0.02   
1     06001400200           0.02           0.02            0.02   
2     06001400300           0.02           0.02            0.02   
3     06001400400           0.02           0.02            0.02   
4     06001400500           0.02           0.02            0.02   
...           ...            ...            ...             ...   
2600  06115040800           0.01           0.01            0.01   
2601  06115040901           0.01           0.

In [44]:
# Run o4
import o4_TractLevel_DamageAssessmentModel as o4
o4_output = o4.build_damage_estimates(o3output, parent_dir)

# -- o5 --

In [45]:
import pandas as pd
import numpy as np

o4out = o4_output.copy()

In [46]:
o4out['GEOID'] = o4out['GEOID'].astype(str)
o4out['GEOID'] = np.where(o4out['GEOID'].str.len() == 11, o4out['GEOID'], o4out['GEOID'].str.zfill(11))
o4out['GEOID'].unique()

array(['06001400100', '06001400200', '06001400300', ..., '06115040902',
       '06115041000', '06115041100'], dtype=object)

In [47]:
from o5_bhi import process_bhi
from o5_svi_module import process_svi
import os
parent_dir = os.path.dirname(os.getcwd())
# ================================================
# o5 - Implement BHI
# ================================================
df = process_bhi(o4out, parent_dir)


df["population"] = df["population"].astype(int)
df["shelter_seeking_low"] = df["BHI_factor_low"]*df["population"]
df["shelter_seeking_high"] = df["BHI_factor_high"]*df["population"]
cols = ["GEOID", "max_intensity", "population", "Total_Num_Building", "risk_level",
        "BHI_factor_low", "BHI_factor_high",
        "shelter_seeking_low", "shelter_seeking_high",
        "Total_Num_Building_Slight", "Total_Num_Building_Moderate", 
            "Total_Num_Building_Extensive", "Total_Num_Building_Complete"]
df = df[cols]
df["GEOID"] = df["GEOID"].astype(str)


# ================================================
# o6 - Download SVI data 
# ================================================
# download SVI (optional)
# process SVI

# apply SVI 
svi = process_svi(parent_dir)
svi["FIPS"] = svi["FIPS"].astype(str)
svi["FIPS"] = np.where(svi["FIPS"].str.len() == 11, svi["FIPS"], svi["FIPS"].str.zfill(11))


df = df.merge(svi, left_on = "GEOID", right_on="FIPS")

# ================================================
# o7 - Combine SVI and BHI, Format Output Data
# ================================================

df["shelter_seeking_low"] = df["shelter_seeking_low"]*df["SVI_Value_Mapped"] 
df["shelter_seeking_high"] = df["shelter_seeking_high"]*df["SVI_Value_Mapped"]

df = df.drop(columns=["FIPS"])

print(df["shelter_seeking_low"].sum())
print(df["shelter_seeking_low"].sum()/df["population"].sum())

df = df.sort_values(by=["shelter_seeking_low"], ascending=False)

print(df.head(50))
save_path = os.path.join(parent_dir, f"Data/final_output_new_{EVENTID}.csv")
df.to_csv(save_path, index=False)

/Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Gradschool/4_SPRING25/Capstone/github/EarthquakeDamageModel_Heinz/WorkingScripts/o5_bhi.py:65: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72) have mixed types. Specify dtype option on import or set low_memory=False.
  pop_data = pd.read_csv(pop_path)


62.95989034854798
6.132855089474769e-06
          GEOID  max_intensity  population  Total_Num_Building risk_level  \
7   06055200501           0.54        5159              1364.0        low   
15  06055200504           0.50        6577              1483.0        low   
30  06095251802           0.34        3132               721.0        low   
14  06055200301           0.50        5602              1433.0        low   
34  06055201004           0.32        6836              1615.0        low   
9   06055200202           0.54        2479               697.0        low   
5   06055200804           0.58        5535              1036.0        low   
18  06055200601           0.48        4892              1461.0        low   
47  06095250701           0.28        3473              1145.0        low   
6   06055200201           0.56        3259              1149.0        low   
3   06055200705           0.58        2845               783.0        low   
8   06055200704           0.54      